In [1]:
import numpy as np
import pandas as pd

Column transformer example

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv("covid_toy.csv")

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["has_covid"]),df['has_covid'],test_size=0.2)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


Fever column has less rows lets start filling the rows

In [34]:
# filling the rows of train data by adding with imputer
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# filling the rows of Test data by adding with imputer
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [35]:
# Ordinal encoding
oe = OrdinalEncoder(categories=[["Mild","Strong"]])
X_train_cough = oe.fit_transform(X_train[['cough']]) 

X_test_cough = oe.fit_transform(X_test[['cough']]) 
X_test_cough.shape

(20, 1)

In [36]:
# Onehot encoding -> gender and city
ohe = OneHotEncoder( drop="first",sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_test_gender_city.shape

(20, 4)

In [45]:
# Drop the age columnas it is not required
X_train_age = X_train.drop(columns = ['gender','fever','city','cough']).values

X_test_age = X_test.drop(columns = ['gender','fever','city','cough']).values

X_test_age.shape

(20, 1)

In [42]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_transformed =  np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# The above things can be minimise using the ColumnTransformer

In [44]:
from sklearn.compose import ColumnTransformer

In [46]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2', OrdinalEncoder(categories=[["Mild","Strong"]]),['cough']),
    ('tnf3', OneHotEncoder( drop="first",sparse=False),['gender','city'])
],remainder = 'passthrough')

In [48]:
transformer.fit_transform(X_train)

array([[103.,   1.,   1.,   0.,   0.,   0.,  46.],
       [ 99.,   0.,   0.,   0.,   0.,   1.,  60.],
       [104.,   0.,   1.,   0.,   0.,   1.,  42.],
       [101.,   1.,   1.,   0.,   1.,   0.,  79.],
       [102.,   0.,   1.,   0.,   0.,   1.,  74.],
       [100.,   0.,   1.,   0.,   1.,   0.,  27.],
       [100.,   0.,   1.,   0.,   0.,   0.,  10.],
       [104.,   0.,   0.,   0.,   0.,   0.,  12.],
       [101.,   0.,   0.,   0.,   1.,   0.,  83.],
       [102.,   0.,   0.,   0.,   0.,   0.,  69.],
       [100.,   0.,   1.,   0.,   0.,   0.,  11.],
       [104.,   0.,   1.,   0.,   0.,   0.,  51.],
       [ 98.,   1.,   1.,   0.,   1.,   0.,  34.],
       [102.,   0.,   0.,   1.,   0.,   0.,  49.],
       [104.,   0.,   0.,   0.,   0.,   0.,  18.],
       [100.,   0.,   1.,   0.,   0.,   0.,  80.],
       [ 98.,   0.,   0.,   0.,   1.,   0.,  31.],
       [104.,   0.,   1.,   0.,   1.,   0.,  16.],
       [101.,   0.,   0.,   0.,   0.,   0.,  20.],
       [104.,   0.,   0.,   0.,